In [ ]:
application "fan";
use application "matroid";

# Cellular (co-)sheaves

A cellular (co-)sheaf on a polyhedral complex $PC$ consists of the following data:

- A vector space $V(P)$ for every polyhedron of $P\in PC$
- A morphism $V(P)\gets V(Q)$ for every face relation $P\le Q\in PC$, or $V(P)\to V(Q)$ for co-sheaves. 

The have to be chosen in a way such that all diagrams commute.

In `polymake` this is realized as an `EdgeMap<Directed, Matrix>` on the Hasse diagram of the polyhedral complex $PC$. The vector spaces are not stored at all, since only their dimension is relevant. Instead just the morphisms are collected, and only for codimension 1 face relations.

We will illustrate this for the example of $PC$ being a square.

In [ ]:
$pc = new PolyhedralComplex(check_fan_objects(new Cone(cube(2))));
print $pc->HASSE_DIAGRAM->ADJACENCY;

We provide several standard constructions, such that one does not have to construct a cellular (co-)sheaf from scratch. One example are the so-called $w$-sheaves, where the vector space associated to a face is the space orthogonal to its affine hull. The $w$-sheaves take an integer indicating which wedge power of the vector spaces should be considered. We will print the matrix of the vector space morphism for every edge in the Hasse diagram, along with auxiliary information for better understanding.

In [ ]:
$G = $pc->HASSE_DIAGRAM->ADJACENCY;
$w1 = $pc->wsheaf(1);
$decor = $pc->HASSE_DIAGRAM->DECORATION;
for(my $edge = entire(edges($pc->HASSE_DIAGRAM->ADJACENCY)); $edge; ++$edge){
    print "Edge: ",$edge->from_node," -- ",$edge->to_node,"\n";
    my $source = $decor->[$edge->from_node]->face;
    my $target = $decor->[$edge->to_node]->face;
    print "corresponds to the face relation ",$source," -- ",$target,"\n";
    my $block = $w1->BLOCKS->[$$edge];
    print "block has ",$block->rows," rows and ",$block->cols," columns\n";
    print $block,"\n";
}

Note that `polymake` inserts an artificial top node for bureaucratic reasons, given as `{-1}`.

# 3-dim cube

For a lattice polytope $\Delta$ and $X$ the toric variety associated to its normal fan there is a result by Danilov that 
$$ H^q(X;\Omega^p) \cong H^q(\Delta;W^p) \otimes_{\mathbb{R}} \mathbb{C} $$

For a simple $\Delta$ and $X$ smooth $H^q(X,\Omega^p) \cong H^{p,q}(X)$ is the Hodge decomposition of the cohomology of $X$.

In [ ]:
$pc = new PolyhedralComplex(check_fan_objects(new Cone(cube(3))));
for(my $i=0; $i<4; $i++){
   my $w = $pc->wcosheaf($i);
   my $cs = $pc->borel_moore_complex($w);
   print topaz::betti_numbers($cs),"\n";
}

## Hodge numbers
The command `hodge_numbers` will compute the Hodge numbers in certain settings. However, for something compact, we only get back the betti numbers of the $f$-cosheaves.

In [ ]:
print hodge_numbers($pc);

# Valuated matroids

For a matroid $M$ the usual homology of the $F$-cosheaves computes the Orlik-Solomon algebra of the matroid. 
This is done by building the $F$-cosheaves on the Bergman fan $B(M)$ of the matroid. When the matorid is connected, it will only have one bounded face, the vertex of the fan. So the cellular chain groups $C^q(B(M),F^p) =0$ unless $q=0$, since only bounded faces are taken into consideration. 

It holds that 
$$ dim(H_0(B(M),F^p)) = dim(OS^p(M))$$
where $OS^p(M)$ is the $p$-th graded part of the Orlik-Solomon alegbra of $M$.

In [15]:
$v = [0,0,3,1,2,1,0,1,0,2,2,0,3,0,4,1,2,2,0,0];
$val_matroid = new matroid::ValuatedMatroid<Min>(BASES=>matroid::uniform_matroid(3,6)->BASES,VALUATION_ON_BASES=>$v,N_ELEMENTS=>6);
$tls = tropical::linear_space($val_matroid);
# Compensate for errors
$tls = new PolyhedralComplex(VERTICES=>$tls->VERTICES, MAXIMAL_POLYTOPES=>$tls->MAXIMAL_POLYTOPES);

@result1 = ();
@result2 = ();
for(my $i=0;$i<3;$i++){
   my $fi = $tls->fcosheaf($i);
   my $si=$tls->usual_chain_complex($fi);
   my $bmi=$tls->borel_moore_complex($fi);
   push @result1, topaz::betti_numbers($si);
   push @result2, topaz::betti_numbers($bmi);
}  
print join("\n", @result1),"\n\n";


1 0 0
5 0 0
10 0 0



# Tropical line

In [ ]:
$f = tropical::toTropicalPolynomial("max(0,x,y,z)"); 
$div = tropical::divisor( (tropical::projective_torus<Max>(3)) , tropical::rational_fct_from_affine_numerator($f)); 
$f0 = $div->fcosheaf(0); 
$f1 = $div->fcosheaf(1); 
$f2 = $div->fcosheaf(2);
$f3 = $div->fcosheaf(3); 
$bm0 = $div->borel_moore_complex($f0); 
$bm1 = $div->borel_moore_complex($f1); 
$bm2 = $div->borel_moore_complex($f2); 
$bm3 = $div->borel_moore_complex($f3); 
print topaz::betti_numbers($bm0),"\n";
print topaz::betti_numbers($bm1),"\n";
print topaz::betti_numbers($bm2),"\n";
print topaz::betti_numbers($bm3),"\n";

In [ ]:
$us0 = $div->usual_chain_complex($f0);
$us1 = $div->usual_chain_complex($f1); 
$us2 = $div->usual_chain_complex($f2);
$us3 = $div->usual_chain_complex($f3); 
print topaz::betti_numbers($us0),"\n";
print topaz::betti_numbers($us1),"\n";
print topaz::betti_numbers($us2),"\n";
print topaz::betti_numbers($us3),"\n";

# Tropical K3 surface

In [ ]:
$f = tropical::toTropicalPolynomial("max(0,x,y,z, 2*x - 2, 2*y-2, 2*z-2, x+y-1, x+z-1, y+z-1, 3*x-6, 3*y-6, 3*z-6, 2*x+y - 4,2*y + x- 4, 2*x + z - 4, 2*z+x - 4, 2*y + z- 4, 2*z+y - 4, x+y+z +1, 4*x -12, 4*y-12, 4*z-12,  3*x+y -9,3*y + x- 9, 3*x + z - 9, 3*z+x - 9, 3*y + z- 9, 3*z+y - 9, 2*x + 2*y - 8, 2*x + 2*z - 8, 2*y + 2*z - 8,2*x + y +z - 7, x + 2*z +y - 7, 2*y + z + x- 7  )");
print $f;

In [ ]:
$k3 = tropical::divisor( (tropical::projective_torus<Max>(3)) , tropical::rational_fct_from_affine_numerator($f));

In [ ]:
$f0 = $k3->fcosheaf(0);
$f1 = $k3->fcosheaf(1);
$f2 = $k3->fcosheaf(2);

In [ ]:
$us0 = $k3->usual_chain_complex($f0);
$us1 = $k3->usual_chain_complex($f1);
$us2 = $k3->usual_chain_complex($f2);
print topaz::betti_numbers($us0),"\n";
print topaz::betti_numbers($us1),"\n";
print topaz::betti_numbers($us2),"\n";

In [ ]:
$bm0 = $k3->borel_moore_complex($f0);
$bm1 = $k3->borel_moore_complex($f1);
$bm2 = $k3->borel_moore_complex($f2);
print topaz::betti_numbers($bm0),"\n";
print topaz::betti_numbers($bm1),"\n";
print topaz::betti_numbers($bm2),"\n";

## Hodge numbers

In [ ]:
print hodge_numbers($k3);